In [ ]:
%load_ext nb_black

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import scanpy as sc
import squidpy as sq
import anndata as ad
import seaborn as sns
import simstpy as sim
import matplotlib.pyplot as plt

In [ ]:
SPATIAL_PATTERNS = [
    "human_DLPFC_151508",
    "human_DLPFC_151674",
    "mouse_cerebellum",
    "mouse_coronal_slices",
    "breast_tumor"
]

In [ ]:
adata = sc.read_h5ad(f"./reference/151507.h5ad")
adata

In [ ]:
library_size_params = sim.rna.fit_library_size(adata.layers["counts"])
gene_mean_params = sim.rna.fit_gene_expression(adata.X, quantile=0.99)

In [ ]:
for i, spatial_pattern in enumerate(SPATIAL_PATTERNS):
    df_spatial = sim.spatial.read_pattern(f"{spatial_pattern}")
    
    sim_adata = sim.rna.sim_multi_group(library_size_params,
                                        gene_mean_params,  
                                        n_svgs=1000,
                                        n_non_svgs=9000,
                                        df_spatial=df_spatial, 
                                        library_id=spatial_pattern,
                                        group_name="spatial_cluster",
                                       fold_change="step")
    
    sim_adata.layers['counts'] = sim_adata.X.copy()
    sc.pp.normalize_total(sim_adata, target_sum=1e4)
    sc.pp.log1p(sim_adata)
    
    sq.pl.spatial_scatter(sim_adata, shape=None, color=["spatial_cluster"], title=spatial_pattern)
    
    sim.rna.compare_library_size(adata.layers['counts'], sim_adata.layers['counts'])
    
    adata2 = sim_adata[:, sim_adata.var.spatially_variable]
    sc.tl.pca(adata2, svd_solver='arpack')
    sc.pp.neighbors(adata2, n_neighbors=10, n_pcs=40)
    sc.tl.umap(adata2)
    sc.pl.umap(adata2, color=['spatial_cluster'], title=f"{spatial_pattern}: UMAP with SVGs")
    
    adata3 = sim_adata[:, ~sim_adata.var.spatially_variable]
    sc.tl.pca(adata3, svd_solver='arpack')
    sc.pp.neighbors(adata3, n_neighbors=10, n_pcs=40)
    sc.tl.umap(adata3)
    sc.pl.umap(adata3, color=['spatial_cluster'], title=f"{spatial_pattern}: UMAP with non-SVGs")
    
    sim_adata.write_h5ad(f"./simulated/{spatial_pattern}.h5ad")